# 1. Metriken

**a)** Führe die folgende Zelle aus, um eine Confusion Matrix zu erhatlten. Bestimme daraus die Prävalenz, den Bias, die Sensitivität und Spezifität, sowie Precision und Recall.

In [ ]:
#@markdown Der Code ist hier nicht weiter interessant, ihr braucht die Zelle einfach nur auszuführen.
import numpy as np
import random as rd
import pandas as pd
from sklearn.metrics import confusion_matrix

rd.seed(0)

fraction_negatives = 0.42
gesamtmenge = 200

y_true=np.concatenate((np.ones((1,int(gesamtmenge*(1-fraction_negatives)))),np.zeros((1,int(gesamtmenge*fraction_negatives)))),1).T
positive_pred_distribution = np.array([min(1,rd.gauss(0.75,0.25)) for i in range(0,int(gesamtmenge*(1-fraction_negatives)))])
negative_pred_distribution = np.array([max(0,rd.gauss(0.33,0.2)) for i in range(0,int(gesamtmenge*fraction_negatives))])
predictions = np.concatenate((positive_pred_distribution,negative_pred_distribution),0)
y_hat = predictions >= 0.66
conf_mat = pd.DataFrame(confusion_matrix(y_true,y_hat), index=["negativ","positiv"],columns=["'negativ' klassifiziert", "'positiv' klassifiziert"])
conf_mat

**b)** Angenommen, ein Klassifikator hat eine Sensitivität von $0.86$, eine Spezifität von $0.85$ und einen F1-Score von $0.12$. Wie groß ist die Prävalenz $p$?


Führe die nächste Zelle aus. Der Code ist absichtlich ausgeblendet, da ihr ihn für diese Übung nicht zu verstehen braucht. Die Zelle erstellt einen Plot, den man interaktiv über einen Schieberegler verändern kann. Der Plot visualisiert das letzte Layer eines Neuronalen Netzes, das eine Klassifikation vornimmt. Das Layer besteht aus einer Sigmoid-Funktion, die die Inputs (repräsentiert durch die senkrechten Striche) in Wahrscheinlichkeiten, also Werte zwischen 0 und 1 umrechnet. Die Farbe der Striche codiert, ob es sich um einen positiven (gelb) oder einen negativen (blau) Wert handelt. Der grüne, wagrechte Strich repräsentiert die Grenze $\alpha$, ab der Werte als positiv klassifiziert werden. Über den Schieberegler kann man $\alpha$ zwischen 0.01 und 0.99 verschieben. 

**c)** Nutze diese Möglichkeit, $\alpha$ verändern zu können, um das optimale $\alpha$ zu schätzen. Skizziere dann per Hand eine PRC- und eine ROC-Kurve. Überlege Dir dafür auch, was in den Grenzfällen $\alpha=0$ und $\alpha=1$ passiert. Gib dann eine zweite Schätzung für den optimalen Wert von $\alpha$, basierend auf deiner Skizze, an.

In [ ]:
#@markdown Ihr müsst diese Zelle nur ausführen, der Code ist nicht weiter interessant
import math
import random
import os

import matplotlib.pyplot as plt

import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout

random.seed(0)
pal = sns.color_palette("colorblind")

def sigmoid(x):
  return 1/(1+np.exp(-x))

def inv_sigmoid(x):
  return np.log(x/(1-x))

x = np.linspace(-10, 10, num=1601)
f1 = sigmoid(x)

def update(alpha):
  plt.figure(figsize=(24,12))
  plt.plot(x,f1,lw=2)
  plt.hlines(alpha, -10, np.log(alpha/(1-alpha)),colors=pal[2],lw=3)
  tp = 10**(-6)
  fp = 10**(-6)
  fn = 10**(-6)
  tn = 10**(-6)
  for p,y in zip(predictions,y_true):
    if y==0:    
      if p < alpha:
        plt.vlines(inv_sigmoid(p),0,p,colors=pal[0],alpha=0.2)
        tn += 1
      else:
        plt.vlines(inv_sigmoid(p),0,p,colors=pal[0])
        fp += 1
    else: 
      if p >= alpha:
        plt.vlines(inv_sigmoid(p),0,p,colors=pal[1])
        tp += 1
      else:
        plt.vlines(inv_sigmoid(p),0,p,colors=pal[1],alpha=0.2)
        fn += 1
  tex = 'TPR: ' + str(int(1000*(tp+10**(-6))/(tp+fn+10**(-6)))/1000) + '\nFPR: ' + str(int(1000*(fp+10**(-6))/(fp+tn+10**(-6)))/1000) + '\nPrecision: ' + str(int(1000*(tp+10**(-6))/(tp+fp+10**(-6)))/1000) + '\nRecall: ' + str(int(1000*(tp+10**(-6))/(tp+fn+10**(-6)))/1000)
  if alpha > 0.8:
    tex_y = 0.6
  else:
    tex_y = 0.8
  plt.text(-8,tex_y,tex,fontsize=20)

interact(update, alpha = widgets.FloatSlider(value=0.5,min=0.005, max=0.995, step = 0.01,layout=Layout(width='1000px')))
plt.show()

# 2. Validierungsverfahren

**a)** Führe die folgende Zelle aus, um die beiden Datensätze *data_for_training.csv* und *data_for_testing.csv* zu importieren. Bei den Daten handelt es sich um einen zufälligen Split eines Datensatzes für [Weinqualität](https://www.kaggle.com/vishalkumbhar1997/wine-quality-prediction-with-logistic-regression). Die Aufgabe besteht darin, einen Klassifikator zu erstellen, der aus den Features die Qualität (`quality`) richtig erkennt. Die Qualität ist dabei für diese Aufgabe binär in gut (1) und schlecht (0) codiert. Führe dann die nächste Zelle aus. Diese erstellt mit der Bibliothek `sklearn` einen Entscheidungsbaum (Der ist das Thema der nächsten Vorlesung, die genaue Funktionsweise ist für diese Übung nicht wichtig), trainiert ihn auf dem Datensatz *data_for_training.csv* und testet ihn dann auf dem Datensatz *data_for_testing.csv*. Wie ihr seht, ist der Score (hier wird standardmäßig die Accuracy berechnet) auf dem zweiten Datensatz deutlich niedriger. Hier liegt also Overfitting vor.

In [ ]:
import pandas as pd
import numpy as np
import random as rd
from google.colab import drive
drive.mount('/content/gdrive')

train_wines = pd.read_csv('/content/gdrive/MyDrive/data_for_training.csv')
test_wines = pd.read_csv('/content/gdrive/MyDrive/data_for_testing.csv')

train_wines

In [ ]:
y = train_wines['quality']
X = train_wines.drop('quality',axis=1)

y_test = test_wines['quality']
X_test = test_wines.drop('quality',axis=1)

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier().fit(X,y)

# training und validation accuracy
print("Tree with depth " + str(tree.get_depth()) + "; train score: "+str(tree.score(X,y)) + "; val score: "+str(tree.score(X_test,y_test)))

**b)** Teile den Datensatz *data_for_training.csv* in einen Trainings- und einen Validierungsdatensatz auf. Der Datensatz *data_for_testing.csv* wird fürs erste nicht weiter beachtet. Nutze die in der Vorlesung vorgestellten Mehtoden der Validierung und Kreuzvalidierung um Hyperparameterkonfigurationen zu finden, die weniger overfitten als die Standardkonfiguration. 

**Hinweis:** Der Entscheidungsbaum hat verschiedene Hyperparamter, unter anderem die maximale Baumtiefe (`max_depth`) und das Teilungskriterium (`criterion`). Man kann zum Beispiel auch spezifizieren, wie viele Parameter bei jeder Teilung berücksichtigt werden sollen (`max_features`). Die Erklärung aller Parameter kann [hier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) nachgelesen werden. Ihr könnt euch bei dieser Aufgabe aber auch auf einen Parameter (z.B. `max_depth`) konzentrieren und für den einen optimalen Wert finden.

**Hinweis2:** Die Bibliothek `sklearn` liefert viele Optionen zur Hyperparameteroptimierung und Validierung. Bei Interesse findet ihr [hier](https://scikit-learn.org/stable/modules/cross_validation.html) und [hier](https://scikit-learn.org/stable/modules/grid_search.html#grid-search) weitere Infos. Die Klasse `sklearn.model_selection.cross_val_score` bietet z.B. die Möglichkeit, unterschiedliche und sogar eigene Gütekriterien bei der Kreuzvalidierung zu berechen. Hier kann man einfach ein bisschen ausprobieren!

**c)** Teste deinen fertigen Klassifikator auf dem bei b) nicht berücksichtigten Datensatz *data_for_testing.csv*. Wie gut ist dein Ergebnis?